In [27]:
'''
Get the transaction history of the ERC20 tokens. 
Author: Jinhua Wang
License: MIT
Powered by Etherscan.io APIs
'''

'\nGet the transaction history of the ERC20 tokens. \nAuthor: Jinhua Wang\nLicense: MIT\nPowered by Etherscan.io APIs\n'

In [28]:
from bs4 import BeautifulSoup
import urllib3
#disable the annoying security warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import sys
import ratel

In [29]:
#the base url for etherscan
baseUrl='https://etherscan.io/'
#the connection pool
pool=urllib3.PoolManager(5)
#number of threads
numthread=5

#get the API key
key=''
keyarr=[]
keyfile='../Credentials/etherscankey.txt'
with open(keyfile) as f:
    keyarr = f.readlines()
if len(keyarr)==0 or keyarr[0] == '':
    print "API Key Not Found in Directory! Aborting ..."
    sys.exit(0)
key=keyarr[0]

#limit the rate that the api could be called
@ratel.rate_limited(5,1)
def call_api(url):
    '''
    Make API calls 
    '''    
    r=pool.request('GET',url)
    if r.status != 200:
        raise ApiError('Cannot call API: {}'.format(response.status_code))
    return response

